In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from os import listdir
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegressionCV
from scipy.sparse import csr_matrix, save_npz, load_npz

from trickster.search import a_star_search, ida_star_search


from defaultcontext import with_default_context
from profiled import Profiler, profiled

MANIFEST_FEATURES = [    # corresponds to...
    'provider',          # "Hardware Components" 
    'permission',        # "Permissions"
    'activity',          # "Components" (185,729 / 218,951)
    'service_receiver',  # "Components" (33,222  / 218,951)
    'intent'             # "Intents"
]                        # ... in the Grosse et al. paper

seed = 2018

In [4]:
# Record hashes corresponding to malware applications
df = pd.read_csv('data/drebin_malware_sha256.csv')
malware_hashes = set(df['sha256'])

In [5]:
data_dir = 'data/drebin/'

# Load the data and record the feature set
data, labels, features = [], [], set()
subset = 50 # use only a subset of the dataset

for file_path in listdir(data_dir)[:subset]:
    with open(data_dir + file_path) as f:
        lines = [x.strip() for x in f]
        if lines == '':
            continue
        data.append(lines)
        features |= set(lines)
        label = 1 if file_path in malware_hashes else 0
        labels.append(label)

In [6]:
# Provide statistics about the feature classes in the DREBIN dataset

classes, seen = {}, set()
for d in data:
    for feature in d:
        if feature in seen:
            continue
        seen.add(feature)
        f = feature.split('::')[0]
        if f == '':
            continue
        classes[f] = classes.get(f, 0) + 1
        
classes_count = sum(classes.values())

for k, v in classes.items():
    print('{}: {}'.format(k, v))
print('Sum of all features: {}.'.format(classes_count))

api_call: 50
feature: 11
url: 484
service_receiver: 22
permission: 51
call: 19
intent: 15
real_permission: 21
activity: 211
provider: 3
Sum of all features: 887.


In [7]:
# Fit a label encoder and transform the input data
label_encoder = LabelEncoder()
label_encoder.fit(list(features))

encoded = []
for i, x in enumerate(data):
    if i != 0 and i % 500 == 0:
        print('Label encoded {} examples.'.format(i))
    e = label_encoder.transform(x)
    encoded.append(e)

# Create a sparse binary matrix from the input data
indptr = np.cumsum([0] + [len(x) for x in encoded])
indices = np.concatenate(encoded)
ones = np.ones(indices.size)

N, K = len(data), len(features)
X = csr_matrix((ones, indices, indptr), shape=(N, K))
y = np.array(labels)
print('Shape of X: {}. Shape of y: {}.'.format(X.shape, y.shape))

Shape of X: (50, 887). Shape of y: (50,).


In [8]:
# Uncomment to save & load data for future use

# save_npz('data/tmp/malware_X.npz', X)
# np.save('data/tmp/malware_y.npy', y)

# X = load_npz('data/tmp/malware_X.npz')
# y = np.load('data/tmp/malware_y.npy')

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((45, 887), (45,), (5, 887), (5,))

In [9]:
# Fit logistic regression and perform CV

Cs = np.arange(0.5, 1.5, 0.025)
class_weight = 'balanced' # balanced or None
scoring = 'accuracy' # accuracy or roc_auc

clf = LogisticRegressionCV(
    Cs=Cs, 
    cv=5, 
    n_jobs=-1, 
    penalty='l2',
    scoring=scoring,
    class_weight=class_weight,
    random_state=seed
)
clf.fit(X_train, y_train)

# Get best score and C value
mean_scores = np.mean(clf.scores_[1], axis=0)
best_idx = np.argmax(mean_scores)
best_score = mean_scores[best_idx]
best_C = clf.Cs_[best_idx]

constant_acc = 1 - sum(y_train) / len(y_train)
print('Training score for a constant model f(x) = 0 is: {:.2f}%.'.format(constant_acc*100))
print('Training accuracy is: {:.2f}%. Best C is: {:.4f}. Class weight: {}. Scoring: {}.'
      .format(clf.score(X_train, y_train)*100, best_C, class_weight, scoring))

# Training score for a constant model f(x) = 0 is: 95.11%.
# Training accuracy is: 99.78%. Best C is: 0.7000. Class weight: balanced. Scoring: accuracy.

Training score for a constant model f(x) = 0 is: 86.67%.
Training accuracy is: 100.00%. Best C is: 1.1500. Class weight: balanced. Scoring: accuracy.


In [10]:
constant_acc = 1 - sum(y_test) / len(y_test)
print('Test score for a constant model f(x) = 0 is: {:.2f}%.'.format(constant_acc*100))
print('Test accuracy is: {:.2f}%.'.format(clf.score(X_test, y_test)*100))

# Test score for a constant model f(x) = 0 is: 95.50%.
# Test accuracy is: 97.40%.

Test score for a constant model f(x) = 0 is: 100.00%.
Test accuracy is: 100.00%.


#### Provide implemention of Algorithm 1 from Grosse et al. paper.

In [11]:
# Provide helper classes and methods
class LogisticRegressionScikitSaliencyOracle:
    def __init__(self, model):
        self.model = model

    def eval(self, _):
        return self.model.coef_[0]
    
def _get_manifest_map(label_encoder):    
    manifest_map = {}
    for i, c in enumerate(label_encoder.classes_):
        feature_class = c.split('::')[0]
        manifest_map[i] = feature_class in MANIFEST_FEATURES
    return manifest_map

In [12]:
# Algorithm 1 from Grosse et al. paper
def find_adversarial_grosse(x, clf, oracle, manifest_map, k=20, return_path=False):
    
    if clf.predict([x]) == 0:
        raise Exception('Initial example is already classified as bening.')
        
    if return_path:
        path = [x]
        
    x_star = np.array(x, dtype='intc')
    distortions = 0
    
    while clf.predict([x_star]) != 0 and distortions < k:
        derivative = oracle.eval(x_star)
        idxs = np.argsort(derivative)
        
        for i, idx in enumerate(idxs):
            
            # Check if changing the feature is permitted.
            if x_star[idx] == 0 and manifest_map[idx]:
                x_star[idx] = 1
                if return_path:
                    path.append(np.array(x_star))
                break
                
            if i == len(idxs) - 1:
                raise Exception('Adversarial example is impossible to create.')
                
        distortions += 1
        
    if distortions == k:
        raise Exception('Distortion bound reached.')
        
    if return_path:
        return x_star, distortions, path
    else:
        return x_star, distortions

In [13]:
oracle = LogisticRegressionScikitSaliencyOracle(clf)
manifest_map = _get_manifest_map(label_encoder) # checks if feature belongs to manifest 

for i, x in enumerate(X):
    
    # Transform the example from compressed matrix format to numpy array.
    x = x.toarray()[0] 
    
    if clf.predict([x]) == 1:
        print('\nCrafting adversarial example for example: {}.'.format(i))
        try:
            x_adv, cost = find_adversarial_grosse(x, clf, oracle, manifest_map, k=100)
        except Exception as e:
            print(e)
            continue
        x_prob, x_adv_prob = clf.predict_proba([x])[0, 1], clf.predict_proba([x_adv])[0, 1]
        print('Start probability: {:.2f}. Resulting probability: {:.2f}. Cost: {}.'.format(x_prob, x_adv_prob, cost))


Crafting adversarial example for example: 0.
Start probability: 0.98. Resulting probability: 0.50. Cost: 59.

Crafting adversarial example for example: 2.
Start probability: 0.97. Resulting probability: 0.49. Cost: 53.

Crafting adversarial example for example: 21.
Start probability: 0.97. Resulting probability: 0.49. Cost: 50.

Crafting adversarial example for example: 23.
Start probability: 0.90. Resulting probability: 0.49. Cost: 25.

Crafting adversarial example for example: 26.
Start probability: 0.97. Resulting probability: 0.49. Cost: 57.

Crafting adversarial example for example: 41.
Start probability: 0.96. Resulting probability: 0.50. Cost: 49.


#### Provide implemention for A* search on the graph.

In [14]:
@with_default_context(use_empty_init=True)
class Counter:
    def __init__(self):
        self.cnt = 0
        
    def increment(self):
        self.cnt += 1
        
    def count(self):
        return self.cnt

In [15]:
# Features that can be altered
manifest_features = [f for (f,b) in manifest_map.items() if b]

class Node:

    def __init__(self, x):
        self.root = x

    def expand(self):
        """Generate all children of the current node."""
        
        # Increment the counter of expanded nodes.
        counter = Counter.get_default()
        counter.increment()
        
        children = []
        
        for feat_idx in manifest_features:
            
            # Skip if the feature is already set.
            if self.root[feat_idx] == 1:
                continue
                
            child = np.array(self.root)
            child[feat_idx] = 1
            children.append(child)
            
        return children
    
    def __repr__(self):
        return 'Node({})'.format(self.root)

In [16]:
def _expand_fn(x, p_norm=1, **kwargs):
    """Wrap the example in `Node`, expand the node, and compute the costs.
    
    Returns a list of tuples (child, cost)
    """
    node = Node(x, **kwargs)
    children = node.expand()
    costs = [np.linalg.norm(x - c, ord=p_norm) for c in children]         
    return list(zip(children, costs))

def _goal_fn(x, clf, target_confidence=0.5):
    """Tell whether the example has reached the goal."""
    return clf.predict_proba([x])[0, 1] <= target_confidence

def _heuristic_fn(x, clf, q_norm=np.inf):
    """Distance to the decision boundary of a logistic regression classifier.
    
    By default the distance is w.r.t. L1 norm. This means that the denominator
    has to be in terms of the Holder dual norm (`q_norm`), so L-inf. I know,
    this interface is horrible.
    
    NOTE: The value has to be zero if the example is already on the target side
    of the boundary.
    """
    score = clf.decision_function([x])[0]
    if score <= 0:
        return 0.0
    h = np.abs(score) / np.linalg.norm(clf.coef_[0, manifest_features], ord=q_norm)    
    return h

def hash_fn(x):
    """Hash function for examples."""
    return hash(x.tostring())

@profiled
def find_adversarial(x, clf, p_norm=1, q_norm=np.inf,
                     target_confidence=0.5, return_path=False, **kwargs):
    """Transform an example until it is classified with target confidence.""" 

    if clf.predict_proba([x])[0, 1] <= target_confidence:
        raise Exception('Initial example is already classified as bening.')        
    return a_star_search(
        start_node=x, 
        expand_fn=lambda x: _expand_fn(x, p_norm=p_norm, **kwargs), 
        goal_fn=lambda x: _goal_fn(x, clf, target_confidence), 
        heuristic_fn=lambda x: _heuristic_fn(x, clf, q_norm=q_norm), 
        hash_fn=hash_fn,
        return_path=return_path
    )

#### Run experiments to compare JSMA with our heuristic.

In [17]:
oracle = LogisticRegressionScikitSaliencyOracle(clf)
manifest_map = _get_manifest_map(label_encoder) # checks if feature belongs to manifest 

for i, x in enumerate(X):
    
    # Transform the example from compressed matrix format to numpy array.
    x = x.toarray()[0] 

    if clf.predict([x]) == 1:
        print('\nCrafting adversarial example for example: {}.'.format(i))
        
        # Try finding adversarial example using JSMA with distortion bound k = 100.
        try:
            x_adv_grosse, cost_grosse = find_adversarial_grosse(x, clf, oracle, manifest_map, k=100)
        except Exception as e:
            print(e)
            continue
            
        # Try finding adversarial example using our heuristic with A* search.
        x_adv, cost = find_adversarial(x, clf, p_norm=1, q_norm=np.inf)
        
        assert clf.predict([x_adv_grosse]) == 0 and clf.predict([x_adv]) == 0
        
        print('Cost using Grosse algorithm: {}. Using our heuristic: {}.'.format(cost_grosse, int(cost)))


Crafting adversarial example for example: 0.
Cost using Grosse algorithm: 59. Using our heuristic: 60.

Crafting adversarial example for example: 2.
Cost using Grosse algorithm: 53. Using our heuristic: 53.

Crafting adversarial example for example: 21.
Cost using Grosse algorithm: 50. Using our heuristic: 50.

Crafting adversarial example for example: 23.
Cost using Grosse algorithm: 25. Using our heuristic: 25.

Crafting adversarial example for example: 26.
Cost using Grosse algorithm: 57. Using our heuristic: 57.

Crafting adversarial example for example: 41.
Cost using Grosse algorithm: 49. Using our heuristic: 50.
